# Reranking Methods in RAG Systems

Origin:
https://colab.research.google.com/github/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/reranking.ipynb

<div style="text-align: center;">

<img src="https://github.com/NirDiamant/RAG_Techniques/blob/main/images/reranking-visualization.svg?raw=1" alt="rerank llm" style="width:100%; height:auto;">
</div>

<div style="text-align: center;">

<img src="https://github.com/NirDiamant/RAG_Techniques/blob/main/images/reranking_comparison.svg?raw=1" alt="rerank llm" style="width:100%; height:auto;">
</div>

# Package Installation and Imports

The cell below installs all necessary packages required to run this notebook.


In [1]:
!pip install -q \
  "numpy<2.0" \
  "langchain==0.1.1" \
  "langchain-core<0.2.0" \
  "langchain-community<0.2.0" \
  langchain-openai \
  langchain-experimental \
  sentence-transformers \
  transformers \
  torch \
  rank_bm25 \
  pymupdf \
  deepeval

In [3]:
# Clone the repository to access helper functions and evaluation modules
!git clone https://github.com/NirDiamant/RAG_TECHNIQUES.git
import sys
sys.path.append('RAG_TECHNIQUES')
# If you need to run with the latest data
# !cp -r RAG_TECHNIQUES/data .

fatal: destination path 'RAG_TECHNIQUES' already exists and is not an empty directory.


**helper_functions.py:**
```
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from pydantic import BaseModel, Field
from langchain import PromptTemplate
from openai import RateLimitError
from typing import List
from rank_bm25 import BM25Okapi
import fitz
import asyncio
import random
import textwrap
import numpy as np
from enum import Enum
```

**evalute.py:**

```
api_key=os.getenv("OPENAI_API_KEY")
base_url=os.getenv("OPENAI_BASE_URL")

llm = ChatOpenAI(
        temperature=0,
        model="gpt-4.1",
        base_url=base_url,
        api_key=api_key,
    )

correctness_metric = GEval(
  name="Correctness",
  model="gpt-4.1",
  evaluation_params=[
    LLMTestCaseParams.EXPECTED_OUTPUT,
    LLMTestCaseParams.ACTUAL_OUTPUT ],
  evaluation_steps=[ "Determine whether the actual output is factually correct based on the expected output." ],
  )

faithfulness_metric = FaithfulnessMetric(
  threshold=0.7,
  model="gpt-4.1",
  include_reason=False
)

relevance_metric = ContextualRelevancyMetric(
  threshold=1,
  model="gpt-4.1",
  include_reason=True
)
```



In [10]:
!pip install langchain-text-splitters

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.2/490.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.5/283.5 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.0.87
    Uninstalling langsmith-0.0.87:
      Successfully uninstalled langsmith-0.0.87
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.23
    Uninstalling langchain-core-0.1.23:
      Successfully uninstalled langchain-core-0.1.23
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.0.6 requires langchain-core<0.2,>=0.1.16, but you have langchain-core 1.2.7 which is incompatible.
langchain-community 0.0.20 requires langchain-core<0.2,>=0.1.21, but you have langchain-core 1.2.7 which is incompatible.
langchain-community 0.0.20 requires langsmith<0.1,>=0.0.83, but you h

In [4]:
import os
import sys
# from dotenv import load_dotenv
# Load environment variables from a .env file
# load_dotenv()

# Set the OpenAI API key environment variable
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('key_ptn')
os.environ["HF_KEY"] = userdata.get('key_hf')
os.environ["OPENAI_BASE_URL"] = "https://llm.ptnglobalcorp.com"

from langchain_core.documents import Document
from typing import List, Dict, Any, Tuple
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_core.retrievers import BaseRetriever
from sentence_transformers import CrossEncoder

# Original path append replaced for Colab compatibility
from helper_functions import *
from evaluation.evalute_rag import *


### Define the document's path

In [5]:
# Download required data files
import os
os.makedirs('data', exist_ok=True)

# Download the PDF document used in this notebook
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf


--2026-01-26 10:00:58--  https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206372 (202K) [application/octet-stream]
Saving to: ‘data/Understanding_Climate_Change.pdf’

data/Understanding_ 100%[===================>] 201.54K  --.-KB/s    in 0.01s   

2026-01-26 10:00:58 (14.2 MB/s) - ‘data/Understanding_Climate_Change.pdf’ saved [206372/206372]

--2026-01-26 10:00:58--  https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185

In [6]:
path = "data/Understanding_Climate_Change.pdf"

In [7]:
llm = ChatOpenAI(
        temperature=0,
        model="gpt-4.1",
        base_url=os.environ["OPENAI_BASE_URL"],
        api_key=os.environ["OPENAI_API_KEY"],
    )
print(llm.invoke("I love programming."))


content='That’s awesome! Programming is a fantastic skill—it lets you build things, solve problems, and express creativity. What languages or projects are you working on, or is there something specific you enjoy most about programming?'


In [8]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 12.0 MB/s eta 0:00:00


### Create a vector store

In [ ]:
!pip install faiss-cpu

In [13]:

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

def replace_t_with_space(list_of_documents):
    """
    Replaces all tab characters ('\t') with spaces in the page content of each document

    Args:
        list_of_documents: A list of document objects, each with a 'page_content' attribute.

    Returns:
        The modified list of documents with tab characters replaced by spaces.
    """

    for doc in list_of_documents:
        doc.page_content = doc.page_content.replace('\t', ' ')  # Replace tabs with spaces
    return list_of_documents

def encode_pdf(path, chunk_size=1000, chunk_overlap=200):
    """
    Encodes a PDF book into a vector store using OpenAI embeddings.

    Args:
        path: The path to the PDF file.
        chunk_size: The desired size of each text chunk.
        chunk_overlap: The amount of overlap between consecutive chunks.

    Returns:
        A FAISS vector store containing the encoded book content.
    """

    # Load PDF documents
    loader = PyPDFLoader(path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    texts = text_splitter.split_documents(documents)
    cleaned_texts = replace_t_with_space(texts)

    # Create embeddings and vector store
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    vectorstore = FAISS.from_documents(cleaned_texts, embeddings)

    return vectorstore

vectorstore = encode_pdf(path)

## Method 1: LLM based function to rerank the retrieved documents

<div style="text-align: center;">

<img src="https://github.com/NirDiamant/RAG_Techniques/blob/main/images/rerank_llm.svg?raw=1" alt="rerank llm" style="width:40%; height:auto;">
</div>

### Create a custom reranking function


In [14]:
class RatingScore(BaseModel):
    relevance_score: float = Field(..., description="The relevance score of a document to a query.")

def rerank_documents(query: str, docs: List[Document], top_n: int = 3) -> List[Document]:
    prompt_template = PromptTemplate(
        input_variables=["query", "doc"],
        template="""On a scale of 1-10, rate the relevance of the following document to the query. Consider the specific context and intent of the query, not just keyword matches.
        Query: {query}
        Document: {doc}
        Relevance Score:"""
    )

    llm = ChatOpenAI(temperature=0, model_name="gpt-4o", max_tokens=4000)
    llm_chain = prompt_template | llm.with_structured_output(RatingScore)

    scored_docs = []
    for doc in docs:
        input_data = {"query": query, "doc": doc.page_content}
        score = llm_chain.invoke(input_data).relevance_score
        try:
            score = float(score)
        except ValueError:
            score = 0  # Default score if parsing fails
        scored_docs.append((doc, score))

    reranked_docs = sorted(scored_docs, key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in reranked_docs[:top_n]]

In [27]:
!pip install langchain

  Using cached langchain_core-0.1.53-py3-none-any.whl.metadata (5.9 kB)
  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_core-0.1.51-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_core-0.1.50-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_core-0.1.49-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_core-0.1.48-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_core-0.1.47-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_core-0.1.46-py3-none-any.whl.metadata (5.9 kB)
INFO: pip is still looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_core-0.1.45-py3-none-any

In [28]:
import re
from typing import List
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# NOTE: Do NOT import PydanticOutputParser or RatingScore here.
# We will use standard Python text processing instead.

def rerank_documents(query: str, docs: List[Document], top_n: int = 3) -> List[Document]:
    """
    Reranks documents based on relevance to the query using a custom LLM model.
    Uses Regex parsing to avoid 'langchain_core.beta' errors.
    """

    # 1. Configure the LLM
    # Make sure to use the exact model name and base_url your API requires
    llm = ChatOpenAI(
        temperature=0,
        model_name="gpt-4.1",       # Your required custom model name
        openai_api_base=os.environ["OPENAI_BASE_URL"],
        api_key=os.environ["OPENAI_API_KEY"],         # Your API Key
        max_tokens=100              # We only need a short answer (the score)
    )

    # 2. Simple Prompt: Ask for a NUMBER only
    prompt_template = PromptTemplate(
        template="""You are a relevance ranking assistant.
        Rate the relevance of the following document to the query on a scale from 0 to 10.

        Query: {query}
        Document: {doc}

        IMPORTANT: Return ONLY a single number (e.g., 8.5). Do not write any explanations.
        Score:""",
        input_variables=["query", "doc"]
    )

    # 3. Create the Chain (Prompt -> LLM)
    chain = prompt_template | llm

    scored_docs = []
    print(f"Scoring {len(docs)} documents using Regex method...")

    for doc in docs:
        try:
            # Invoke the chain
            response = chain.invoke({"query": query, "doc": doc.page_content})

            # Extract content safely (Handles different LangChain response objects)
            if hasattr(response, 'content'):
                content = response.content
            else:
                content = str(response)

            # 4. ROBUST PARSING: Find the number using Regex
            # This looks for patterns like "8.5", "10", "Score: 7" inside the text
            match = re.search(r"[-+]?\d*\.\d+|\d+", content)

            if match:
                score = float(match.group())
            else:
                score = 0.0 # Default to 0 if no number found

            # Save score to metadata
            doc.metadata["rerank_score"] = score
            scored_docs.append((doc, score))

        except Exception as e:
            print(f"Error processing document: {e}")
            scored_docs.append((doc, 0.0))

    # 5. Sort by score (Highest first)
    reranked_docs = sorted(scored_docs, key=lambda x: x[1], reverse=True)

    # Return top N documents
    return [doc for doc, _ in reranked_docs[:top_n]]


# --- TEST BLOCK ---
# Run this immediately to verify the fix
if 'initial_docs' in locals() and len(initial_docs) > 0:
    print("Testing rerank_documents...")
    try:
        final_docs = rerank_documents(query, initial_docs)
        print("\n--- Success! Top Document ---")
        print(f"Score: {final_docs[0].metadata.get('rerank_score')}")
        print(final_docs[0].page_content[:150] + "...")
    except Exception as e:
        print(f"Still failing: {e}")
else:
    print("Please run the 'initial_docs = vectorstore.similarity_search(...)' step first.")

Testing rerank_documents...
Scoring 15 documents using Regex method...

--- Success! Top Document ---
Score: 10.0
Climate change is altering terrestrial ecosystems by shifting habitat ranges, changing species 
distributions, and impacting ecosystem functions. Fore...


### Example usage of the reranking function with a sample query relevant to the document


In [29]:
query = "What are the impacts of climate change on biodiversity?"
initial_docs = vectorstore.similarity_search(query, k=15)
reranked_docs = rerank_documents(query, initial_docs)

# print first 3 initial documents
print("Top initial documents:")
for i, doc in enumerate(initial_docs[:3]):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:200] + "...")  # Print first 200 characters of each document


# Print results
print(f"Query: {query}\n")
print("Top reranked documents:")
for i, doc in enumerate(reranked_docs):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:200] + "...")  # Print first 200 characters of each document

Scoring 15 documents using Regex method...
Top initial documents:

Document 1:
Climate change is altering terrestrial ecosystems by shifting habitat ranges, changing species 
distributions, and impacting ecosystem functions. Forests, grasslands, and deserts are 
experiencing shi...

Document 2:
protection, and habitat creation. 
Climate-Resilient Conservation 
Conservation strategies must account for climate change impacts to be effective. This 
includes identifying climate refugia, areas le...

Document 3:
The economic costs of climate change include damage to infrastructure, reduced agricultural 
productivity, health care costs, and lost labor productivity. Extreme weather events, such as 
hurricanes a...
Query: What are the impacts of climate change on biodiversity?

Top reranked documents:

Document 1:
Climate change is altering terrestrial ecosystems by shifting habitat ranges, changing species 
distributions, and impacting ecosystem functions. Forests, grasslands, and deserts are

### Create a custom retriever based on our reranker

In [34]:
# Create a custom retriever class
from pydantic import BaseModel, Field

class CustomRetriever(BaseRetriever):

    vectorstore: Any = Field(description="Vector store for initial retrieval")

    class Config:
        arbitrary_types_allowed = True

    def get_relevant_documents(self, query: str, num_docs=2) -> List[Document]:
        initial_docs = self.vectorstore.similarity_search(query, k=30)
        return rerank_documents(query, initial_docs, top_n=num_docs)


# Create the custom retriever
custom_retriever = CustomRetriever(vectorstore=vectorstore)

# Create an LLM for answering questions
llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-4.1",
    openai_api_base=os.environ["OPENAI_BASE_URL"],
    api_key=os.environ["OPENAI_API_KEY"])

# Create the RetrievalQA chain with the custom retriever
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=custom_retriever,
    return_source_documents=True
)


### Example query


In [35]:
result = qa_chain({"query": query})

print(f"\nQuestion: {query}")
print(f"Answer: {result['result']}")
print("\nRelevant source documents:")
for i, doc in enumerate(result["source_documents"]):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:200] + "...")  # Print first 200 characters of each document

/usr/local/lib/python3.12/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Scoring 30 documents using Regex method...

Question: What are the impacts of climate change on biodiversity?
Answer: Climate change impacts biodiversity in several significant ways:

1. **Shifting Habitat Ranges:** As temperatures rise and precipitation patterns change, many species are forced to move to new areas where conditions are more suitable. This can lead to changes in the geographic ranges of both plants and animals.

2. **Changing Species Distributions:** Some species may expand their ranges, while others may contract or even disappear from certain areas. This alters the composition of ecosystems and can disrupt existing ecological relationships.

3. **Loss of Biodiversity:** The inability of some species to adapt or migrate quickly enough can lead to population declines and extinctions, reducing overall biodiversity.

4. **Disruption of Ecosystem Functions:** Changes in species composition can affect ecosystem services such as pollination, nutrient cycling, and water regula

### Example that demonstrates why we should use reranking

In [36]:
chunks = [
    "The capital of France is great.",
    "The capital of France is huge.",
    "The capital of France is beautiful.",
    """Have you ever visited Paris? It is a beautiful city where you can eat delicious food and see the Eiffel Tower.
    I really enjoyed all the cities in france, but its capital with the Eiffel Tower is my favorite city.""",
    "I really enjoyed my trip to Paris, France. The city is beautiful and the food is delicious. I would love to visit again. Such a great capital city."
]
docs = [Document(page_content=sentence) for sentence in chunks]


def compare_rag_techniques(query: str, docs: List[Document] = docs) -> None:
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    vectorstore = FAISS.from_documents(docs, embeddings)

    print("Comparison of Retrieval Techniques")
    print("==================================")
    print(f"Query: {query}\n")

    print("Baseline Retrieval Result:")
    baseline_docs = vectorstore.similarity_search(query, k=2)
    for i, doc in enumerate(baseline_docs):
        print(f"\nDocument {i+1}:")
        print(doc.page_content)

    print("\nAdvanced Retrieval Result:")
    custom_retriever = CustomRetriever(vectorstore=vectorstore)
    advanced_docs = custom_retriever.get_relevant_documents(query)
    for i, doc in enumerate(advanced_docs):
        print(f"\nDocument {i+1}:")
        print(doc.page_content)


query = "what is the capital of france?"
compare_rag_techniques(query, docs)

Comparison of Retrieval Techniques
Query: what is the capital of france?

Baseline Retrieval Result:

Document 1:
The capital of France is huge.

Document 2:
The capital of France is great.

Advanced Retrieval Result:
Scoring 5 documents using Regex method...

Document 1:
The capital of France is great.

Document 2:
The capital of France is beautiful.


## Method 2: Cross Encoder models

<div style="text-align: center;">

<img src="https://github.com/NirDiamant/RAG_Techniques/blob/main/images/rerank_cross_encoder.svg?raw=1" alt="rerank cross encoder" style="width:40%; height:auto;">
</div>

### Define the cross encoder class

In [38]:
from typing import List, Any
from langchain_core.retrievers import BaseRetriever
from langchain_core.documents import Document
from langchain_core.pydantic_v1 import Field # Use LangChain's internal Pydantic

# 1. Inherit ONLY from BaseRetriever
class CrossEncoderRetriever(BaseRetriever):

    vectorstore: Any = Field(description="Vector store for initial retrieval")
    cross_encoder: Any = Field(description="Cross-encoder model for reranking")
    k: int = Field(default=5, description="Number of documents to retrieve initially")
    rerank_top_k: int = Field(default=3, description="Number of documents to return after reranking")

    class Config:
        arbitrary_types_allowed = True

    # 2. Rename to _get_relevant_documents (note the underscore)
    # The run_manager argument is required by the base class signature in newer versions
    def _get_relevant_documents(self, query: str, *, run_manager=None) -> List[Document]:
        # Initial retrieval from vector store
        initial_docs = self.vectorstore.similarity_search(query, k=self.k)

        # Basic check to avoid errors if no docs found
        if not initial_docs:
            return []

        # Prepare pairs for cross-encoder (Query, Document Text)
        pairs = [[query, doc.page_content] for doc in initial_docs]

        # Get cross-encoder scores
        scores = self.cross_encoder.predict(pairs)

        # Zip docs with scores and sort
        scored_docs = sorted(zip(initial_docs, scores), key=lambda x: x[1], reverse=True)

        # Return top K reranked documents
        return [doc for doc, _ in scored_docs[:self.rerank_top_k]]

# --- Usage ---

# Initialize the CrossEncoder
# (Make sure you have sentence-transformers installed)
from sentence_transformers import CrossEncoder
cross_encoder_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# Create the retriever
retriever = CrossEncoderRetriever(
    vectorstore=vectorstore,
    cross_encoder=cross_encoder_model,
    k=10,             # Fetch 10 candidates
    rerank_top_k=3    # Return best 3
)

print("CrossEncoderRetriever created successfully!")

CrossEncoderRetriever created successfully!


### Create an instance and showcase over an example

In [40]:
# Create the cross-encoder retriever
cross_encoder_retriever = CrossEncoderRetriever(
    vectorstore=vectorstore,
    cross_encoder=cross_encoder,
    k=10,  # Retrieve 10 documents initially
    rerank_top_k=5  # Return top 5 after reranking
)

# Set up the LLM
llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-4.1",
    openai_api_base=os.environ["OPENAI_BASE_URL"],
    api_key=os.environ["OPENAI_API_KEY"])

# Create the RetrievalQA chain with the cross-encoder retriever
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=cross_encoder_retriever,
    return_source_documents=True
)

# Example query
query = "What are the impacts of climate change on biodiversity?"
result = qa_chain({"query": query})

print(f"\nQuestion: {query}")
print(f"Answer: {result['result']}")
print("\nRelevant source documents:")
for i, doc in enumerate(result["source_documents"]):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:200] + "...")  # Print first 200 characters of each document


Question: What are the impacts of climate change on biodiversity?
Answer: Climate change has significant impacts on biodiversity, affecting both terrestrial and marine ecosystems. Here are the main impacts:

**1. Shifts in Habitat Ranges and Species Distributions:**  
Climate change causes many species to move to new areas as temperatures and precipitation patterns change. This can lead to shifts in habitat ranges and changes in where species are found.

**2. Changes in Species Composition:**  
Forests, grasslands, and deserts are experiencing changes in the types of plants and animals that live there. Some species may thrive, while others decline or disappear, leading to a loss of biodiversity and disruption of ecological balance.

**3. Disruption of Ecosystem Functions:**  
As species distributions and compositions change, the functions that ecosystems provide—such as pollination, water purification, and carbon storage—can be disrupted.

**4. Loss of Biodiversity:**  
The combined e

![](https://europe-west1-rag-techniques-views-tracker.cloudfunctions.net/rag-techniques-tracker?notebook=all-rag-techniques--reranking)